In [ ]:
!pip install --upgrade google-api-python-client

In [2]:
from googleapiclient.discovery import build

In [78]:
import json

<h5 align='right'>Youtube API Keys and Authorization</h5>
<hr color='f47b0a'>

In [16]:
apikey='XXXX'

In [17]:
ytb=build('youtube', 'v3', developerKey=apikey)


<h5 align='right'>Selecting the RedBull channel</h5>
<hr color='f47b0a'>

In [18]:
channel_id='UCblfuW_4rakIf2h6aqANefA'
channel_name = 'RedBull'

<h5 align='right'>Collecting information</h5>
<hr color='f47b0a'>

In [116]:
def get_info(ytb, channel_id):
  req = ytb.channels().list(
      part = 'snippet, brandingSettings, contentDetails, statistics',
      id=channel_id)
  res=req.execute()
  return res['items'] #to see all the option take out the []

In [117]:
channel_stts = get_info(ytb, channel_id)

In [118]:
len(channel_stts)

1

In [121]:
channel_stts[0]['snippet']

{'country': 'AT',
 'customUrl': 'redbullgivesyouwings',
 'description': 'Experience the world of Red Bull like you have never seen it before, with the best action sports clips and original series on YouTube.\n\nWatch highlights from sport events like UCI Mountain Biking, Red Bull Rampage, Red Bull Ice Cross, Red Bull Straight Rhythm, Red Bull Soap Box Race, Red Bull Flugtag, Gaming, Volcom Pipe Pro, Red Bull Cliff Diving or discover the behind the scenes to athletes life from all over the globe.\n\nYou can watch plenty more on http://redbull.com ',
 'localized': {'description': 'Experience the world of Red Bull like you have never seen it before, with the best action sports clips and original series on YouTube.\n\nWatch highlights from sport events like UCI Mountain Biking, Red Bull Rampage, Red Bull Ice Cross, Red Bull Straight Rhythm, Red Bull Soap Box Race, Red Bull Flugtag, Gaming, Volcom Pipe Pro, Red Bull Cliff Diving or discover the behind the scenes to athletes life from all ov

In [120]:
channel_stts[0]['brandingSettings']

{'channel': {'country': 'AT',
  'description': 'Experience the world of Red Bull like you have never seen it before, with the best action sports clips and original series on YouTube.\n\nWatch highlights from sport events like UCI Mountain Biking, Red Bull Rampage, Red Bull Ice Cross, Red Bull Straight Rhythm, Red Bull Soap Box Race, Red Bull Flugtag, Gaming, Volcom Pipe Pro, Red Bull Cliff Diving or discover the behind the scenes to athletes life from all over the globe.\n\nYou can watch plenty more on http://redbull.com ',
  'keywords': '"red bull" redbull "action sports" "extreme sports" 4k stratos "space jump" gopro downhill rampage "red bull rampage" "crashed ice" "frozen rush" "straight rhythm" "ryan sheckler" "travis pastrana" "danny macaskill" "drew benazon" "Kriss Kyle" "red bull signature series" BMX Motocross skateboarding snowboarding "mountain biking" "sky diving" surfing skiing rallycross music F1 motorbike moto "formula 1" f1 POV soapbox flugtag skydive parachute',
  'tit

In [119]:
channel_stts[0]['statistics']

{'hiddenSubscriberCount': False,
 'subscriberCount': '10300000',
 'videoCount': '5723',
 'viewCount': '2554745445'}

In [39]:
channel_stts[0]['contentDetails']

{'relatedPlaylists': {'likes': '', 'uploads': 'UUblfuW_4rakIf2h6aqANefA'}}

In [105]:
playlist = channel_stts[0]['contentDetails']['relatedPlaylists']['uploads']

In [100]:
playlist

'UUblfuW_4rakIf2h6aqANefA'

<h5 align='right'>Collecting videos info by playlist</h5>
<hr color='f47b0a'>

In [123]:
def get_video(ytb, playlist):
  video_list =[]
  req = ytb.playlistItems().list(
      part='snippet, contentDetails',
      playlistId=playlist,
      maxResults = 50
  )

  next_page = True
  while next_page:
    res = req.execute()
    data=res['items']

    for video in data:
      video_id = video['contentDetails']['videoId']
      if video_id not in video_list:
        video_list.append(video_id)

    if 'nextPageToken' in res.keys():
      next_page = True
      req=ytb.playlistItems().list(
        part='snippet, contentDetails',
        playlistId=playlist,
        maxResults = 50,
        pageToken=res['nextPageToken']
      )
    else:
      next_page = False

  return video_list


In [53]:
video_list = get_video(ytb, playlist)

In [124]:
len(video_list)

5723

<h5 align='right'>Collecting video statistics</h5>
<hr color='f47b0a'>

In [125]:
def get_video_stts(ytb, video_list):
  stts_list = []

  for i in range(0, len(video_list),50):
    req = ytb.videos().list(
        part='id,snippet, contentDetails, statistics',
        id=video_list[i:i+50]
    )
    data=req.execute()

    for video in data['items']:
            stats_dict=dict(video)
            stts_list.append(stats_dict)
  return stts_list

In [88]:
video_data = get_video_stts(ytb, video_list)

In [126]:
len(video_data)

5723

In [89]:
video_data[2
           ]

{'contentDetails': {'caption': 'false',
  'contentRating': {},
  'definition': 'hd',
  'dimension': '2d',
  'duration': 'PT2M6S',
  'licensedContent': True,
  'projection': 'rectangular'},
 'etag': 'fpaPdZpqU7QWwCKoI3QuyAiVeuA',
 'id': 'jxe870HmYBE',
 'kind': 'youtube#video',
 'snippet': {'categoryId': '17',
  'channelId': 'UCblfuW_4rakIf2h6aqANefA',
  'channelTitle': 'Red Bull',
  'defaultAudioLanguage': 'en',
  'defaultLanguage': 'en',
  'description': 'Formula 1 World Champion Max Verstappen enjoyed a chilled start to 2022 as he put some laps in on an ice track against Austrian Speedway legend Franky Zorn. On his first drive back in an F1 car in 2022 with the number 1 on his helmet, Max pitted for some special spiked tyres and had some fun on the GP Ice Race circuit. \n\nExplore more of what Red Bull Racing have done with F1 cars: https://www.redbull.com/int-en/shows/red-bull-racing-road-trips \n\n\n#RedBull #F1 #MaxVerstappen\n_________________________________________________\nExpe

In [90]:
len(video_data)

5723

<h5 align='right'>Save the data</h5>
<hr color='f47b0a'>

In [93]:
with open('/content/drive/MyDrive/Colab Notebooks/video_data.json', 'w') as fp:
  json.dump(video_data, fp, indent=4)